In [341]:
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from operator import itemgetter
import json

In [313]:
order_food = {  
    "name" : "Order Food on Uber",
    "api_call": "uber.eat.order",
    "description": "Order food on uber eat given a list of items and the quantity of items respectively",
    "parameters": 
        [
            {
                "name": "restaurants", 
                "description": "The restaurants user wants to order from" 
            }, 
            {
                "name": "items", 
                "description": "A list of order user wants to order from restaurants"
            },
            {
                "name": "quantities", 
                "description": "A list of quantities corresponding to the items ordered"
            }
        ]
    }

In [314]:
ride_share = {
        "name": "Uber Carpool",
        "api_name": "uber.ride",
        "description": "Find suitable ride for customers given the location, type of ride, and the amount of time the customer is willing to wait as parameters",
        "parameters":  [
            {"name": "loc", "description": "Location of the starting place of the Uber ride"},
            {"name": "type", "enum": ["plus", "comfort", "black"], "description": "Types of Uber ride user is ordering"},
            {"name": "time", "description": "The amount of time in minutes the customer is willing to wait"}
        ]
    }

In [355]:
functions_string = json.dumps([order_food,ride_share])

In [316]:
gorilla_llm = Ollama(model="gorilla-openfunctions", temperature=0)

In [318]:
mistral_llm = Ollama(model="mistral", temperature=0)

In [357]:
gorilla_prompt = PromptTemplate.from_template("USER: <<question>> {question} <<function>> {functions_string}\nASSISTANT: ").partial(functions_string=functions_string)

In [358]:
mistral_prompt = PromptTemplate.from_template("""<s>[INST]You are a helpful AI API assistant. You have access to the following tools:

{functions_string}

Only use these tools to generate appropriate calls based on user's questions. Use parameter name based on the tool description.

For example:
Order a ride to the airport of type comfort. I want to leave in 10 minutes.
I want to order 2 big macs and a regular fries from McDonalds and a diet coke from 711.
I am craving chicken biriyani from bawarchi.
I need to leave for work in 20 mins. 
[/INST]</s>
uber.ride(loc="airport", type="comfort", time=10)
uber.eat.order(restaurants="McDonalds", items=["big macs","regular fries"], quantities=[2,1])
uber.eat.order(restaurants="711", items=["diet coke"], quantities=[1])
uber.eat.order(restaurants="bawarchi", items=["chicken biriyani"], quantities=[1])
uber.ride(loc="work", type="comfort", time=20)
[INST]
{question}
[/INST]
""").partial(functions_string=functions_string)

In [408]:
prompt = PromptTemplate.from_template("{request}")

In [409]:
chain = (
    prompt | 
    {
        "question": RunnablePassthrough() | RunnableLambda(lambda p: p.to_string()), 
        "functions_string": RunnableLambda(lambda x:functions_strings)
    } | 
    {
        "gorilla": gorilla_prompt | gorilla_llm | StrOutputParser(), 
        "mistral": mistral_prompt | mistral_llm | StrOutputParser() 
    }
)

In [410]:
chain.invoke({"request":"Order 2 big macs from McDonalds"})

{'gorilla': 'uber.eat.order(restaurants="McDonalds", items=["2 big macs"], quantities=[2])',
 'mistral': 'uber.eat.order(restaurants="McDonalds", items=["big macs"], quantities=[2])'}

In [411]:
chain.invoke({"request":"Book a ride to the airport within next 10 minutes"})

{'gorilla': 'uber.ride(loc="airport", type="comfort", time=10)',
 'mistral': 'uber.ride(loc="airport", type="comfort", time=10)'}

In [412]:
chain.invoke({"request":"I am craving a regular philly cheesestake with a diet coke and chips from Jersey Mike's"})

{'gorilla': 'uber.eat.order(restaurants=["Jersey Mike\'s"], items=["Philly Cheesestake", "Diet Coke", "Chips"], quantities=[1, 1, 1])',
 'mistral': 'uber.eat.order(restaurants="Jersey Mike\'s", items=["regular philly cheesestake","diet coke","chips"], quantities=[1,1,1])\nuber.ride(loc="home", type="comfort", time=20)'}

In [413]:
chain.invoke({"request":"I want to order five burgers and six chicken wings from McDonlad and go to the airport in 10 minutes"})

{'gorilla': 'uber.eat.order(restaurants="McDonald\'s", items=["5 burgers", "6 chicken wings"], quantities=[10])',
 'mistral': 'uber.eat.order(restaurants="McDonalds", items=["burger", "chicken wings"], quantities=[5,6])\nuber.ride(loc="airport", type="comfort", time=10)'}

In [414]:
chain.invoke({"request":"I need a ride to be at the park in 15 minutes"})

{'gorilla': 'uber.ride(loc="park", type="comfort", time=15)',
 'mistral': 'uber.ride(loc="park", type="comfort", time=15)'}

In [415]:
chain.invoke({"request":"I want to order chicken tikka masala, 2 naans, and 1 chicken biriyani from Mirchi and an El Diablo burger and fries from Hopdoddys"})

{'gorilla': 'uber.eat.order(restaurants=["Mirchi", "Hopdoddys"], \n              items=["chicken tikka masala", "2 naans", "1 chicken biriyani"], \n              quantities=[1, 2, 3])',
 'mistral': 'uber.eat.order(restaurants=["Mirchi", "Hopdoddy\'s"], items=["chicken tikka masala", "2 naans", "1 chicken biriyani", "El Diablo burger", "fries"], quantities=[1, 2, 1, 1, 1])\nuber.ride(loc="home", type="comfort", time=30)'}